In [1]:
import json
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [6]:
nltk.download('punkt')
nltk.download('stopwords')

# Load the intents JSON file
with open('intent.json') as file:
    data = json.load(file)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Preprocessing function
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    return " ".join(tokens)

In [8]:
# Prepare data for training
patterns = []
labels = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(preprocess(pattern))
        labels.append(intent['tag'])

# Convert to DataFrame
df = pd.DataFrame({'text': patterns, 'label': labels})
df

,text,label
0,problem bill,billing
1,charged extra,billing
2,explain bill,billing
3,app working,technical_support
4,ca log account,technical_support
5,website,technical_support
6,ca access account,account_issues
7,account hacked,account_issues
8,reset password,account_issues


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to TF-IDF features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['text'])
y = df['label']

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Evaluate the model
# y_pred = model.predict(X_test)
# print("Accuracy:", accuracy_score(y_test, y_pred))

LogisticRegression()

In [18]:
def predict_intent(text):
    # Preprocessing the input text
    text = preprocess(text)
    # Converting to TF-IDF features
    text_vec = vectorizer.transform([text])
    # Predicting the intent
    intent = model.predict(text_vec)[0]
    # finding a random response for the predicted intent
    for item in data['intents']:
        if item['tag'] == intent:
            return np.random.choice(item['responses'])

In [21]:
# Test the model with a sample input
sample_input = "my bill is not showing"
print("Predicted Response:", predict_intent(sample_input))

Predicted Response: Please provide your billing details so we can assist you.
